In [34]:
import numpy as np
from ase import Atoms
from copy import deepcopy
from itertools import combinations
import pandas as pd
from ase.io import read, write

In [22]:
from ase.visualize import view

In [23]:
from ase.build import molecule

In [24]:
cn = molecule('CN')
cn.positions

array([[ 0.      ,  0.      , -0.611046],
       [ 0.      ,  0.      ,  0.523753]])

In [25]:
cn.positions
cn.set_cell([10,10,10])
cn.pbc = True
cn.center()
cn.rotate(v='x',a=np.pi/2., center='COM')

In [26]:
view(cn)

In [27]:
def build_lattice( points, motif, lattice_constant ):
    "A function for creating very simple lattices"
    motifs_list = []
    # A lattice with two points
    if points == 2:
        for point in range(points):
            this_motif = motif.copy()
            if point == 0:
                motifs_list.append(this_motif)
            else:
                this_motif.translate([lattice_constant,0.0,0.0])
                motifs_list.append(this_motif)
    # A lattice with three points
    if points == 3:
        for point in range(points):
            this_motif = motif.copy()
            if point == 0:
                motifs_list.append(this_motif)
            elif point == 1:
                this_motif.translate([-lattice_constant/2.0,-lattice_constant*np.sin(np.pi/3.0),0.0])
                motifs_list.append(this_motif)
            else:
                this_motif.translate([lattice_constant/2.0,-lattice_constant*np.sin(np.pi/3.0),0.0])
                motifs_list.append(this_motif)
    if points == 4:
        for point in range(points):
            this_motif = motif.copy()
            if point == 0:
                motifs_list.append(this_motif)
            elif point == 1:
                this_motif.translate([lattice_constant,0.0,0.0])
                motifs_list.append(this_motif)
            elif point == 2:
                this_motif.translate([0.0,-lattice_constant,0.0])
                motifs_list.append(this_motif)
            elif point == 3:
                this_motif.translate([lattice_constant,-lattice_constant,0.0])
                motifs_list.append(this_motif)
    return motifs_list


In [28]:
lattice = build_lattice(4, motif=cn,lattice_constant=3)

In [10]:
lattice

[Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...),
 Atoms(symbols='CN', pbc=True, cell=[10.0, 10.0, 10.0], magmoms=...)]

In [11]:
conf = Atoms()

In [12]:
for point in lattice:
    conf.extend(point)

In [13]:
view(conf)

In [15]:
def create_configurations( lattice, angle ):
    """A function for generating all possible combinations of a list
    of configurations. The result is a dictionary with the label 'A'
    being used to mark positions on the lattice where no rotation has
    been applied and the label 'C' to indicate that this point has undergone
    a rotation of 180.0 about the z-axis.
    """
    les_confs = []
    conf_names = []
    for L in range(len(lattice)+1):
        for subset in combinations(range(len(lattice)), L):
            name = len(lattice) * ['A']
            lattice_copy = deepcopy(lattice)
            conf = Atoms()
            if len(subset) == 0:       
                for item in lattice_copy:
                    conf.extend(item)
            elif len(subset) == len(lattice_copy):
                for item in lattice_copy:
                    item.rotate(v='z',a=np.pi, center='COM')
                    conf.extend(item)
                    name = len(lattice) * ['C']
            else:
                for index in subset:
                    lattice_copy[index-1].rotate(v='z',a=np.pi, center='COM')
                    name[index] = 'C'
                for item in lattice_copy:
                    conf.extend(item)
            les_confs.append(conf)
            conf_names.append(''.join(name))
    return dict(zip(conf_names,les_confs))

In [16]:
confs = create_configurations(lattice,180.0)

In [17]:
confs

{'AAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'AACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'ACCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CAAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CACC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCAC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCA': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...),
 'CCCC': Atoms(symbols='CNCNCNCN', pbc=False, magmoms=...)}

In [18]:
view(confs.values())

In [29]:
df = pd.DataFrame(columns=confs.keys(),index=confs.keys())

In [30]:
df

,CACA,CCAA,ACAC,CAAC,CCCA,AAAC,ACCC,AACC,CACC,AAAA,ACCA,CCAC,ACAA,CAAA,CCCC,AACA
CACA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df['AACA']['CCAA'] = 2.0

In [32]:
df

,CACA,CCAA,ACAC,CAAC,CCCA,AAAC,ACCC,AACC,CACC,AAAA,ACCA,CCAC,ACAA,CAAA,CCCC,AACA
CACA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
ACAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CACC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
bmim_pf6_opt = read('./opt_coords_bigbox.xyz')
bmim_pf6_opt.cell = [44.20800018, 42.54000092, 50.]
bmim_pf6_opt.center()

In [49]:
view(bmim_pf6_opt)

In [40]:
pf6 = bmim_pf6_opt[0:7]; bmim = bmim_pf6_opt[7:32]; electrode = bmim_pf6_opt[32:]

In [53]:
pair = pf6+bmim
z_shift = abs(electrode.get_center_of_mass()-pair.get_center_of_mass())[2]
z_shift

4.409996147134045

In [46]:
view(pair)

In [54]:
electrode.center()
pair.center()
pair.translate([0.0,0.0,-z_shift])

In [55]:
view(pair)

In [64]:
lattice = build_lattice(4, motif=pair,lattice_constant=11.0)

In [65]:
confs = create_configurations(lattice,180.0)

In [66]:
confs

{'AAAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'AAAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'AACA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'AACC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACCA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'ACCC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CAAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CAAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CACA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CACC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCAA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCAC': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCCA': Atoms(symbols='C32H60F24N8P4', pbc=False),
 'CCCC': Atoms(symbols='C32H60F24N8P4', pbc=False)}

In [68]:
view(confs.values())

In [69]:
confs['AAAA']

Atoms(symbols='C32H60F24N8P4', pbc=False)